In [1]:
def savings(cls):
    cls.account_type = 'savings'
    return cls

def checking(cls):
    cls.account_type = 'checking'
    return cls



In [2]:
class Account:
    pass

@savings
class Bank1Savings(Account):
    pass

@savings
class Bank2Savings(Account):
    pass

@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass



In [3]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'savings'})

In [4]:
Bank2Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'checking'})

In [5]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

In [6]:
@account_type("Savings")
class Bank1Savings:
    pass

In [7]:
@account_type("Checking")
class Bank1Checking:
    pass

In [8]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Savings' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Savings' objects>,
              '__doc__': None,
              'account_type': 'Savings'})

In [9]:
Bank1Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Checking' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Checking' objects>,
              '__doc__': None,
              'account_type': 'Checking'})

In [10]:
def hello(cls):
    cls.hello = lambda self: f'{self} says hello!'
    return cls

In [11]:
@hello
class Person:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

In [12]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [13]:
p = Person('Guido')
p.hello()

'Guido says hello!'

In [3]:
from functools import wraps

def func_logger(fn):

    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f"Log: {fn.__qualname__}({args}, {kwargs}) = {result}")
        return result
    return inner

In [18]:
class Person:
    @func_logger
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @func_logger
    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years old.'

In [19]:
p = Person('Tho', 80)

Log: Person.__init__((<__main__.Person object at 0x000001F26520C350>, 'Tho', 80), {}) = None


In [20]:
p.greet()

Log: Person.greet((<__main__.Person object at 0x000001F26520C350>,), {}) = Hello, my name is Tho, and I am 80 years old.


'Hello, my name is Tho, and I am 80 years old.'

In [2]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating: ", cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [26]:
@class_logger
class Person:

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years old.'

decorating:  <class '__main__.Person'> __init__
decorating:  <class '__main__.Person'> greet


In [27]:
p = Person('Tho', 20)

Log: Person.__init__((<__main__.Person object at 0x000001F2654B4320>, 'Tho', 20), {}) = None


In [28]:
p.greet()

Log: Person.greet((<__main__.Person object at 0x000001F2654B4320>,), {}) = Hello, my name is Tho, and I am 20 years old.


'Hello, my name is Tho, and I am 20 years old.'

In [ ]:
@class_logger
class Person:
    
    @staticmethod
    def static_method():
        print("staticmethod invoked...")

    @classmethod
    def cls_method(cls):
        print(f"cls_metho invoked for {cls}...")
    
    def instance_method(self):
        print(f"instance_method invoked for {self}...")

decorating:  <class '__main__.Person'> static_method
decorating:  <class '__main__.Person'> instance_method


In [30]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              'static_method': <function __main__.Person.static_method()>,
              'cls_method': <classmethod(<function Person.cls_method at 0x000001F2659879C0>)>,
              'instance_method': <function __main__.Person.instance_method(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [32]:
callable(Person.__dict__["static_method"])

True

In [4]:
class Person:
    
    @staticmethod
    @func_logger
    def static_method():
        pass


In [34]:
Person.static_method()

Log: Person.static_method((), {}) = None


In [5]:
class Person:
    
    @func_logger
    @staticmethod
    def static_method():
        pass


In [6]:
Person.static_method()

Log: Person.static_method((), {}) = None


In [7]:
class Person:
    @staticmethod
    def static_method(self):
        pass

    @classmethod
    def class_method(self):
        pass

In [ ]:
type(Person.__dict__['static_method'])

True

In [ ]:
type(Person.__dict__['class_method'])

False

In [13]:
Person.__dict__['static_method'].__func__


<function __main__.Person.static_method(self)>

In [16]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating callable", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print("decorating static method", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, decorated_func, method)
        elif isinstance(obj, classmethod):
            print(f"decorating class method", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
    return cls

In [17]:
@class_logger
class Person:
    @staticmethod
    def static_method(a, b):
        print('static_method called...', a, b)
    
    @classmethod
    def class_method(cls, a, b):
        print("class_method called...", a, b)
    
    def instance_method(self, a, b):
        print("instance_method called...", a, b)

decorating callable <class '__main__.Person'> static_method
decorating class method <class '__main__.Person'> class_method
decorating callable <class '__main__.Person'> instance_method


In [18]:
Person.static_method(10, 20)

static_method called... 10 20
Log: Person.static_method((10, 20), {}) = None


In [19]:
Person.class_method(10, 20)

class_method called... 10 20
Log: Person.class_method((<class '__main__.Person'>, 10, 20), {}) = None


In [20]:
p = Person()
p.instance_method(10, 20)

instance_method called... 10 20
Log: Person.instance_method((<__main__.Person object at 0x0000021F34388CE0>, 10, 20), {}) = None


In [21]:
@class_logger
class Person:
    def __ini__(self, name):
        self._name = name
    
    @property
    def name(self):
        return self._name 

decorating callable <class '__main__.Person'> __ini__


In [22]:
type(Person.__dict__['name'])

property

In [23]:
isinstance(Person.__dict__['name'], property)

True

In [24]:
prop = Person.__dict__['name']

In [25]:
prop.fget

<function __main__.Person.name(self)>

In [26]:
prop.fget, prop.fdel

(<function __main__.Person.name(self)>, None)

In [27]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating callable", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print("decorating static method", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, decorated_func, method)
        elif isinstance(obj, classmethod):
            print(f"decorating class method", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print("decorating property", cls, name)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
    return cls